サンプルソース
-----------------

https://github.com/thinkitcojp/TensorFlowDL-samples

TensorFlowとは
---------------
- Define & Run形式
- 分散のプラットフォームができている
- サーバーで使うときはdockerを使うと便利(nvidia-docker)

Define & runのメリット
---------------
- リソースの効率化
- 実行計画の最適化ができる

In [3]:
import tensorflow as tf

### tf.Variableとtf.constant
- Variableは変数
- constantは定数

### サンプル

In [28]:
# constantは定数
a = tf.constant(3, name="const1")
# Variableは変数(objectも可)
# 名前を指定しなくてもよい
b = tf.Variable(1, name="val1")
# 加算
add = tf.add(a, b)
# 変数bに代入
# assignは代入する関数
assign = tf.assign(b, add)
# プレースホルダ
c = tf.placeholder(tf.int32, name="input")
#掛け算
mul = tf.multiply(assign, c)
# 変数の初期化
# 多分これで変数名と代入するキーの紐付けをしてるのでは?
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(mul, feed_dict={c:2}))
    # aも入れることができる
    print(sess.run(mul, feed_dict={a:2, c:2}))

8
12
18
24


In [22]:
a = tf.constant(3, name='const1')
b = tf.Variable(0, name='val1')
add = tf.add(a, b)
c = tf.placeholder(tf.int32, name="input")
assign = tf.assign(b, add)
mul = tf.multiply(assign, c)

init = tf.global_variables_initializer() #変数の初期化！！
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(mul, feed_dict={c:1}))
    print(sess.run(mul, feed_dict={c:1})) #assignで変数bに値が蓄積されていくので値が変わっていく
    print(sess.run(mul, feed_dict={c:1}))

3
6
9


## テンソルボード

In [24]:
#インライン　テンソルボード
from IPython.display import clear_output, Image, display, HTML
import numpy as np

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
    
#セッション上の計算グラフをクリア
tf.reset_default_graph()
#=====================
h1 = tf.placeholder(tf.int32)
h2 = tf.placeholder(tf.int32)
# add_op = h1+h2
#5+1=6
add_op = tf.add(h1, h2)
var1 = tf.Variable(20) 
#update_var1 = add_op
#var1 = add_op
update_var1 = tf.assign(var1, add_op) 
#6*6
# mul_op = add_op * update_var1
mul_op = tf.multiply(add_op, update_var1)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    result = sess.run(mul_op, feed_dict={h1:5, h2:1})
    print(result)
    show_graph(tf.get_default_graph().as_graph_def()) # show graph

Instructions for updating:
Use `tf.global_variables_initializer` instead.
36


### tf.placeholder()
- 宣言時には値が分からないもの

```
assign = tf.assign(b, add)
c = tf.placeholder(tf.int32, name="input")
mul = tf.mul(d, e)
```

- 初期化時に型だけ決めておき実行時にdictionary型で実際の値を与える

### 変数の初期化
- tf.global_variables_initializerで初期化する

### tf.Session()で実行する


### tf.reduce_mean()
- テンソルの平均値を計算する
- numpyのnp.mean()と同じ
- 第２引数にaxisをとる

In [24]:
import tensorflow as tf

a = [[1., 2., 3.], [4., 5., 6.]]

x = tf.placeholder(tf.float32, [2, 3])
y_1 = tf.reduce_mean(x, axis=0)
y_2 = tf.reduce_mean(x, axis=1)
y_3 = tf.reduce_mean(x)

with tf.Session() as sess:
    print(a)
    print(sess.run(y_1, feed_dict={x: a}))
    print(sess.run(y_2, feed_dict={x: a}))
    print(sess.run(y_3, feed_dict={x: a}))

[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]
[2.5 3.5 4.5]
[2. 5.]
3.5


- axis(軸)が0だと列に対して平均値を出す
- axis(軸)が1だと行に対して平均値を出す
- axis(軸)が指定なしだと全ての値の平均値をだす

MNISTを使った入門
----------------------

In [31]:
import pandas as pd
# mnistはtensorflowにあるexampleを使う
# (バイナリからベクトルに変換するのが大変なので)
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [32]:
# mnistデータを格納したオブジェクトを呼び出す
# 第一引数で指定したディレクトリにダウンロードされる
# one_hotとは0か1かどちらかのデータ
mnist = input_data.read_data_sets("data/", one_hot=True)

# 全訓練データの取得

# 訓練用の入力データ、正解データをミニバッチ数を指定して取得
train_images, train_labels = mnist.train.next_batch(50)

# テスト用の全画像データを取得
test_images = mnist.test.images
# テスト用の全正解データを取得
test_labels = mnist.test.labels

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


### 順伝播

In [33]:
# 入力データを定義

# モデル構築時にどの画像を渡すか決められないので、tf.placeholder()を使って実行時に引数で渡す
# 画素値は正規化してあるため浮動小数点=>float32
# 入力は[ミニバッチサイズ、１枚の画像の画素数]
## Noneに指定するのは、入力時に自動で計算するため値はなんでもいいという意味
x = tf.placeholder(tf.float32, [None, 784])

In [34]:
# 入力層から中間層

# 中間層のユニット数は64にする
# 64という数字は、経験則
# 層が多すぎる場合は、過学習になる(drop_out)
# 学習パラメタ、バイアスともに確率勾配法を用いてアップデートされる変数なのでtf.Variableで宣言
w_1 = tf.Variable(tf.truncated_normal([784, 64], stddev=0.1), name="w1")
# バイアスはしきい値
b_1 = tf.Variable(tf.zeros([64]), name="b1")
h_1 = tf.nn.relu(tf.matmul(x, w_1) + b_1)

- truncated_normalは正規分布をもとに指定したshapeの値を自動生成するオペレーション
    - ニューラルネットワークの重みを初期化するときには基本的にこれを使用する
- tf.Variable(tf.zeros([64]), name="b1")
    - バイアスは一般的に0で初期化する
    - shapeが合わないがブロードキャスティング機能で演算されるため問題ない
        - ※ブロードキャスティング機能: shapeが合わないテンソル同士の演算で階層が多い方のshapeに合わせて上手く演算してくれる
- tf.matmul(x, w_1)で行列の積を出しReLUにかける

In [35]:
# 中間層から出力層

w_2 = tf.Variable(tf.truncated_normal([64, 10], stddev=0.1), name="w2")
b_2 = tf.Variable(tf.zeros([10]), name="b2")
out = tf.nn.softmax(tf.matmul(h_1, w_2) + b_2)

### 誤差関数、訓練

In [36]:
# 正解ラベルも実行時に渡せるようにplaceholderにする
y = tf.placeholder(tf.float32, [None, 10])
# 二乗誤差関数を採用
loss = tf.reduce_mean(tf.square(y - out))

In [40]:
# 訓練
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

- GradientDescentOptimizerは確率的勾配降下法の実装オペレーション
    - 引数の0.5は学習率を表す
    - 学習率は非常に重要

In [41]:
# 評価
# 一番高い数値の配列のindexが等しければ正解とみなしている
correct = tf.equal(tf.argmax(out, 1), tf.argmax(y, 1))
# castでfloatに変換している
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

- tf.argmaxで正解データと出力データで一番大きな値を求める
    - 引数1はaxisのため行に対して最大値を求めている
- tf.equalで同じラベルを指しているかを判定する
- Trueの割合を求めることで正解率を出している

In [42]:
# 変数の初期化
init = tf.global_variables_initializer()

### 実行

In [45]:
with tf.Session() as sess:
    sess.run(init)
    
    #　テストデータのロード
    test_images = mnist.test.images
    test_labels = mnist.test.labels
    
    for i in range(1000):
        step = i + 1
        train_images, train_labels = mnist.train.next_batch(50)
        # 学習
        sess.run(train_step, feed_dict={x: train_images, y: train_labels})
       
        # 10stepごとに評価もする
        if step % 10 == 0:
            acc_val = sess.run(accuracy, feed_dict={x: test_images, y: test_labels})
            print("Step {:d}: accuracy = {:.2}".format(step, acc_val))

Step 10: accuracy = 0.11
Step 20: accuracy = 0.16
Step 30: accuracy = 0.21
Step 40: accuracy = 0.27
Step 50: accuracy = 0.33
Step 60: accuracy = 0.37
Step 70: accuracy = 0.39
Step 80: accuracy = 0.42
Step 90: accuracy = 0.43
Step 100: accuracy = 0.45
Step 110: accuracy = 0.46
Step 120: accuracy = 0.47
Step 130: accuracy = 0.48
Step 140: accuracy = 0.5
Step 150: accuracy = 0.51
Step 160: accuracy = 0.53
Step 170: accuracy = 0.55
Step 180: accuracy = 0.59
Step 190: accuracy = 0.61
Step 200: accuracy = 0.63
Step 210: accuracy = 0.65
Step 220: accuracy = 0.65
Step 230: accuracy = 0.68
Step 240: accuracy = 0.69
Step 250: accuracy = 0.72
Step 260: accuracy = 0.73
Step 270: accuracy = 0.74
Step 280: accuracy = 0.75
Step 290: accuracy = 0.76
Step 300: accuracy = 0.77
Step 310: accuracy = 0.78
Step 320: accuracy = 0.8
Step 330: accuracy = 0.8
Step 340: accuracy = 0.81
Step 350: accuracy = 0.82
Step 360: accuracy = 0.82
Step 370: accuracy = 0.82
Step 380: accuracy = 0.82
Step 390: accuracy = 0.8

Tenserbord
------------------
- 構築した計算グラフを可視化するためのツール

### Tenserbordで取得できるログ
- 精度などのスカラー値の推移を可視化したScalars
- 入力や加工された画像データを可視化するImages
- 音声情報を扱うAudio
- 構築された計算グラフを可視化するGraphs
- 重みなどの分布情報の推移を可視化するHistograms/Distributions
- 高次元データを空間上に埋め込んで可視化するEmbedding
- 文章情報を扱うText

### TensorBordでログを見るまでのプロセス
- モデル構築時に必要なところでログを取得するオペレーションを記載
- 全てログを配置し終わったら、そのログをマージするオペレーションを記載
- `tf.summary.FileWriter`を呼び出してログの出力先と対象グラフを指定する
- ログを取得するds